# NFL Tendency Analysis

We have had plenty of papers address when to go for it on 4th down, or the most optimal play to call on 3rd down. In this notebook, we will explore tendency analysis across the league and individual teams.

This notebook will go through the methodology of data collection, analysis, and results through multiple lenses. Once I explore the play-by-play data I have and any augmentations to the data, I will determine next steps and perhaps find a way to find some insights on the history of plays called given a game situation (i.e. 1st down, 2nd and short, 2nd and long, 3rd and short, 3rd and long, 4th). This will be quantified by the target variable of whether the play was a run or pass.

*We perhaps may narrow this analysis down to only passing plays in 2018 (given available data). If this is the new focus, then we will consider how offensive / defensive personnel matchups affect the target variable of yards gained per play.*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)

## Data Scraping via nflfastR Package

We collect data for the past decade, 2010-2020. This is an arbitrary amount, but it reflects the shift of the NFL to a more pass-heavy league. Perhaps in subsamples of the data this will hold even more true, since there are considerable rule amendments that can affect the style of play each season. I used the following R code to pull the data. It was incredibly fast and easy to obtain via the nflFastR package and saved lots of time in scraping.

https://www.nflfastr.com/articles/nflfastR.html

```
require(nflfastR)
decade <- load_pbp(2010:2020)
write.csv(decade, file='nfl_pbp_2010-2020.csv')
```

This file is imported below, and is 983 MB. There is a considerable amount of data that can be chopped down to reduce the dimensionality of the data as well.

In [2]:
pbp_init = pd.read_csv('nfl_pbp_2010-2020.csv')
pbp_init = pbp_init.drop('Unnamed: 0', axis=1)

C:\Users\tshah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,214,215,219,220,221,223,225,227,234,235,236,237,238,239,249,250,254,255,256,261,263,264,267,268,269,270,284,285,295,296,302) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pbp_init

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

We are provided with 372 columns in this data. We can supplement this dataset with offensive and defensive rankings for each team, win and score probability features, and PFF (Pro Football Focus) grades. I do not want to pay for a PFF subscription, so I will hold that thought for now and perhaps return to it later if fitted models do not perform well or could use additional information.

Before we do that, though, we must clean and augment our original data. First, we remove all features relating to individual player names or statistics on the play, since the intent is to do a team-based analysis. We also will look to remove post-snap features. Any information relating to the outcome of each play will defeat the purpose of our analysis when validating the models and thus the model will be useless if implemented in practice.

In [10]:
# after perusing columns, I figured these were most relevant to analysis
to_keep = ['play_id', 'game_id', 'posteam', 'posteam_type', 'defteam',
           'drive', 'qtr', 'down', 'time', 'yardline_100', 'ydstogo',
           'desc', 'play_type', 'shotgun', 'no_huddle',
           'posteam_score', 'defteam_score', 'score_differential',
           'no_score_prob', 'fg_prob', 'safety_prob', 'td_prob',
           'wp', 'def_wp']
pbp = pbp_init[to_keep]
pbp

,play_id,game_id,posteam,posteam_type,defteam,drive,qtr,down,time,yardline_100,ydstogo,desc,play_type,shotgun,no_huddle,pass_length,pass_location,air_yards,run_location,run_gap,posteam_score,defteam_score,score_differential,no_score_prob,fg_prob,safety_prob,td_prob,wp,def_wp
0,1,2010_01_ARI_STL,NaN,NaN,NaN,NaN,1,NaN,15:00,NaN,0,GAME,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.422024,0.577976
1,36,2010_01_ARI_STL,ARI,away,LA,1.0,1,NaN,15:00,30.0,0,3-Josh.Brown kicks 70 yards from STL 30 to ARI...,kickoff,0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.004885,0.198696,0.003165,0.323388,0.422024,0.577976
2,58,2010_01_ARI_STL,ARI,away,LA,1.0,1,1.0,14:55,78.0,10,(14:55) 3-D.Anderson pass short right to 83-S....,pass,0,0,short,right,0.0,NaN,NaN,0.0,0.0,0.0,0.004996,0.203153,0.003407,0.323469,0.433909,0.566091
3,82,2010_01_ARI_STL,ARI,away,LA,1.0,1,2.0,14:24,78.0,10,(14:24) 34-T.Hightower left end to ARI 27 for ...,run,0,0,NaN,NaN,NaN,left,end,0.0,0.0,0.0,0.004976,0.186679,0.003384,0.289591,0.415989,0.584011
4,103,2010_01_ARI_STL,ARI,away,LA,1.0,1,3.0,13:43,73.0,5,(13:43) (Shotgun) 3-D.Anderson pass short righ...,pass,1,0,short,right,7.0,NaN,NaN,0.0,0.0,0.0,0.005572,0.194630,0.003174,0.282675,0.403295,0.596705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531510,4280,2020_21_KC_TB,KC,away,TB,21.0,4,2.0,01:40,10.0,6,"(1:40) (No Huddle, Shotgun) 15-P.Mahomes pass ...",pass,1,1,short,right,10.0,NaN,NaN,9.0,31.0,-22.0,0.207054,0.220636,0.001242,0.541650,0.001261,0.998739
531511,4307,2020_21_KC_TB,TB,home,KC,22.0,4,1.0,01:33,80.0,10,(1:33) 12-T.Brady kneels to TB 19 for -1 yards.,qb_kneel,0,0,NaN,NaN,NaN,NaN,NaN,31.0,9.0,22.0,1.000000,0.000000,0.000000,0.000000,0.999959,0.000041
531512,4328,2020_21_KC_TB,TB,home,KC,22.0,4,2.0,00:50,81.0,11,(:50) 12-T.Brady kneels to TB 19 for no gain.,qb_kneel,0,0,NaN,NaN,NaN,NaN,NaN,31.0,9.0,22.0,1.000000,0.000000,0.000000,0.000000,0.999946,0.000054
531513,4349,2020_21_KC_TB,TB,home,KC,22.0,4,3.0,00:30,81.0,11,(:30) 12-T.Brady kneels to TB 18 for -1 yards.,qb_kneel,0,0,NaN,NaN,NaN,NaN,NaN,31.0,9.0,22.0,1.000000,0.000000,0.000000,0.000000,0.999933,0.000067


In [17]:
# remove play types that are not "pass" or "run"
pbp['play_type'].unique()

array([nan, 'kickoff', 'pass', 'run', 'field_goal', 'punt', 'no_play',
       'extra_point', 'qb_kneel', 'qb_spike'], dtype=object)

# Potential Outcomes

- Predict play type (run/pass) to predict playcalling tendencies
    - Goal is to see how predictable a team is with their play calling
    - Does being predictable in play calling (i.e. if a machine can learn their tendencies) lead to fewer wins?
    - Quantify how advantageous it is to be creative
    - Would need to fit models to each team?
    - Utilize data from 2021 NFL Big Data Bowl
        - https://www.kaggle.com/c/nfl-big-data-bowl-2021/data
        - https://www.kaggle.com/c/nfl-big-data-bowl-2020/data
    - More in-depth analysis in separate notebook

# Features to Include

The main features I think will be useful in this model from the 2010-2020 PBP data:

- Type of play (run/pass): TARGET
- Drive number
- Quarter of the game
- Down number
- Time left in a quarter
- Yards to gain for a first down
- Yards to gain for a touchdown (100 minus current yardline position)
- One-hot encoded formation: shotgun / no-huddle
- Current score in the game (for the possessing team and defensive team, which would be modeled as a difference)
- Debate whether to keep in:
    - Probability of drive ending in no score
    - Probability of drive ending in FG
    - Probability of drive ending in safety
    - Probability of drive ending in TD
    - Win probability of offensive team
    - Win probability of defensive team
- Offensive Team
- Defensive Team
- Week of season

Depending on the outcome I want to focus on, some of these may not be relevant.

# Models

Depends on continous or categorical outcome..

# Preliminary Results